# Analyse de l'Évolution de la Vulgarité dans les Chansons par Genre Musical


L'analyse présentée dans ce notebook explore l'évolution de la vulgarité dans les chansons de différents genres musicaux à travers le temps. Le dataset utilisé provient de [Hugging Face](https://huggingface.co/spaces/alihmaou/parolesparolesparoles/resolve/main/genius_top_50_fr_rap_pop_rock_analyzed.parquet), qui contient des données analysées sur des chansons issues du top 50 français dans les genres rap, pop et rock. Ce dataset enrichi comprend des informations telles que le titre de la chanson, l'artiste, l'année de sortie, la complexité lexicale, les thèmes principaux, la tonalité émotionnelle, ainsi que des informations géographiques sur le lieu d'écriture de la chanson.

L'objectif principal de cette analyse est de générer une visualisation qui illustre l'évolution de la vulgarité par type de musique, en incluant un nuage de points représentant les chansons dispersées aléatoirement sur un jour de publication dans l'année. Cette visualisation permettra d'observer les tendances dans la vulgarité des chansons au fil du temps, tout en identifiant les spécificités de chaque genre musical.


## Méthodologie

La méthodologie utilisée dans cette analyse implique plusieurs étapes clés. Premièrement, les données sont chargées et nettoyées pour s'assurer que seules les chansons avec des années de sortie valides sont incluses dans l'analyse. Ensuite, une date aléatoire dans l'année est générée pour chaque chanson pour simuler la dispersion des chansons sur une année. Les données sont ensuite jointes avec les moyennes de vulgarité annuelles calculées pour chaque type de musique. Finalement, une visualisation interactive est créée à l'aide de Plotly pour illustrer l'évolution de la vulgarité par type de musique, ainsi que la dispersion des chansons individuelles en fonction de leur vulgarité et de leur date de publication.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [1]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://huggingface.co/spaces/alihmaou/parolesparolesparoles/resolve/main/genius_top_50_fr_rap_pop_rock_analyzed.parquet", safe_mode=True)
print("✅ Données chargées avec succès")

## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [2]:
# Exécution de la requête
df = run_query(""" WITH 
-- Générer une date aléatoire dans l'année pour chaque chanson
chanson_info AS (
 SELECT 
 "titre" AS chanson,
 "artiste",
 "vulgarite",
 "style_musique" AS type_musique,
 CAST("annee" AS VARCHAR) AS annee_str,
 -- Nettoyer et générer une date aléatoire dans l'année
 CASE 
 WHEN LENGTH(CAST("annee" AS VARCHAR)) = 4 THEN CAST("annee" AS VARCHAR) || LPAD(CAST((RANDOM() * 12 + 1) AS INTEGER)::VARCHAR, 2, '0') || LPAD(CAST((RANDOM() * 28 + 1) AS INTEGER)::VARCHAR, 2, '0')
 ELSE NULL
 END AS jour_publication
 FROM 
 loaded_dataset
 WHERE 
 LENGTH(CAST("annee" AS VARCHAR)) = 4 -- Filtrer les années incorrectes
),
-- Calculer la moyenne de vulgarité par année et par type de musique
avg_vulgarite AS (
 SELECT 
 "style_musique",
 CAST("annee" AS VARCHAR) AS annee_str,
 AVG(CAST("vulgarite" AS DOUBLE)) AS moyenne_vulgarite_annuelle
 FROM 
 loaded_dataset
 WHERE 
 LENGTH(CAST("annee" AS VARCHAR)) = 4 -- Filtrer les années incorrectes
 GROUP BY 
 "style_musique",
 "annee"
)
SELECT 
 CAST(ci.jour_publication AS INTEGER) AS jour_publication,
 ci.chanson,
 ci.artiste,
 ci.vulgarite,
 av.moyenne_vulgarite_annuelle,
 ci.type_musique
FROM 
 chanson_info ci
JOIN 
 avg_vulgarite av ON ci.type_musique = av.style_musique AND ci.annee_str = av.annee_str
WHERE 
 ci.jour_publication IS NOT NULL -- Exclure les enregistrements avec des dates nulles
ORDER BY 
 ci.jour_publication """)
print(f"Résultats : {len(df)} lignes")
df.head()

## 📈 Visualisation

La bibliothèque principale utilisée est Plotly, qui est une technologie adaptée pour créer des visualisations interactives et dynamiques. Plotly permet de représenter les données de manière claire et détaillée, avec des fonctionnalités de zoom et de survol pour une analyse approfondie. Cela convient parfaitement pour représenter l'évolution de la vulgarité par type de musique au fil des ans.

In [3]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

df['jour_publication'] = pd.to_datetime(df['jour_publication'].astype(str), format='%Y%m%d', errors='coerce')
df = df.dropna(subset=['jour_publication'])
df['annee'] = df['jour_publication'].dt.year
df['annee_jitter'] = df['annee'] + np.random.uniform(-0.2, 0.2, size=len(df))

palette = px.colors.qualitative.Set2
types = sorted(df['type_musique'].dropna().unique())
couleurs = {t: palette[i % len(palette)] for i, t in enumerate(types)}

dataviz = go.Figure()

for t in types:
    df_type = df[df['type_musique'] == t]
    df_moy = df_type.groupby('annee')['moyenne_vulgarite_annuelle'].mean().reset_index()
    dataviz.add_trace(
        go.Scatter(
            x=df_moy['annee'], y=df_moy['moyenne_vulgarite_annuelle'],
            mode='lines', name=f"{t} (moy annuelle)",
            line=dict(color=couleurs[t], width=3),
            hovertemplate='%{y:.2f}<extra>%{fullData.name}</extra>'
        )
    )
    dataviz.add_trace(
        go.Scatter(
            x=df_type['annee_jitter'], y=df_type['vulgarite'],
            mode='markers', name=t,
            marker=dict(color=couleurs[t], size=5, opacity=0.3),
            customdata=df_type[['chanson', 'artiste', 'jour_publication', 'type_musique']],
            hovertemplate='%{customdata[0]}<br>%{customdata[1]}<br>%{customdata[2].dt.strftime("%d/%m/%Y")}<br>%{y}<extra></extra>',
            showlegend=False
        )
    )

dataviz.update_layout(
    title="Évolution de la vulgarité par type de musique",
    xaxis_title="Année",
    yaxis_title="Vulgarité",
    template="plotly_white",
    margin=dict(l=20, r=20, t=50, b=20),
    height=550,
    legend_title="Style"
)
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*